# Criptografia RSA
Como toda criptografia, a RSA é um método para troca de mensagens entre dois indivíduos (digamos, Alice e Bob), sem que outos indivíduos consigam compreender o conteúdo da mensagem.

## Função $ \varphi $ de Euller
Lembrando aqui que a função $\varphi$ de Euller é uma função definida como:

$$
\varphi(n) = |\left\{a \in \left\{1, ..., n\right\} | \ mdc(a, \ n) = 1\right\}|
$$

Ou seja, é a quantidade de números entre 1 e n que são coprimos com n.
É derivável deste teorema a seguinte informação:
* Sejam $p$ e $q$ dois primos. Assim:

$$
\varphi(pq) = (p - 1)(q-1)
$$

## Criptografando, parte 1:
Para criptografar uma mensagem via RSA, vamos seguir a seguinte sequência de passos

1. Bob escolhe $n$ tal que $n$ é produto de dois primos, $p$ e $q$. Note também que $n$ é um número grande.

2. Bob também escolhe um número $e \in \left\{2,...,n-1\right\}$ tal que $mdc(e, \varphi(n)) = 1$.
  * Logicamente, isso implica que $e$ é inversível módulo $\varphi(n)$
  * Isso vai significar que existe $d\ |\ ed\ \equiv\ 1\ mod\ (\varphi(n)) $

3. Bob, então, torna público o par $(n, e)$. Esta é conhecida como a **Chave Pública**
  * Toda pessoa que desejar enviar uma mensagem sigilosa para Bob terá acesso a este par. 
  * Bob guarda o par $(\varphi(n), d)$ em sigilo, portanto esta é conhecida como **Chave Privada** 
  



## Criptografando, parte 2:
1. Alice deseja enviar uma mensagem, mais especificamente, um númerom, para Bob.
  * O número que Alice quer enviar, $b$, fica entre $1$ e $n$.
  * Alice calcula então $C(b) = b^e\ mod\ n = c$
    
2. Alice, então, envia $c$ para Bob.

3. Bob, tendo recebido $c$, segue para calcular $D(c) = c^d\ mod\ n = d$
  * Neste caso, $D(c)$ é a mensagem descodificada.

## Bob escolhe os primos e faz a Chave Pública

In [2]:
def codifica_bloco(mesg:str, size=20) -> list:
  """Função para codificar uma mensagem de acordo com o tamanho do Bloco"""
  
  bloco, n =  0, 0
  lista = []
  l_final = []
  
  while(n + size <= len(mesg)):
    lista.append(mesg[n:n + size])
    n += size
    
  if len(mesg)%size != 0:
    blocos_criados = (len(mesg) // size)
    lista.append(mesg[(blocos_criados * size):])

  for i in lista:
    mes = i
    
    for idx, s in enumerate(mes):
      bloco += ord(s) * 256**idx 
      
    l_final.append(bloco)
    bloco = 0
    
  return l_final


def decodifica_bloco(num):
  """Função para descodificar uma mensagem em blocos."""
  b = []

  for i in num:
    a = []
    c = []
    
    while i > 0:
      a.append(i%256)
      i = i//256

    
    for j in a:
      c.append(chr(j))
    b.append(str(c).strip().replace(',', '').replace('[', '').replace(']', ' ').replace("'", ''))
    
  return b

# Aqui serão definidas funções

def XMDC(a, b):
  """Algoritmo Extendido de Euclides, para cálculo das equações modulares"""
  x0, x = 1, 0
  y0, y = 0, 1
  
  sign_a, sign_b = 1,  1
  if a < 0:
    a = -a
    sign_a = -1
  
  if b < 0:
    b = -b
    sign_b = -1
    
  while b > 0:
    q, r = a//b, a%b
    a, b = b, r
    
    x, x0 = x0 - q * x, x
    y, y0 = y0 - q * y, y
    
  return a, sign_a * x0, sign_b * y0

def ExpModN(a, e, n):
  """Calcula a^e mod n"""
  A, P, E = a, 1, e
  while E != 0:
    D = E % 2
    
    if D == 1:
      P = (A*P) % n
      E = (E-1)//2
      
    else:
      E = E//2
      
    A = (A*A) % n
    
  return P

def inverso_modular(a, n):
  """Calcula o inverso modular de a em Zn"""
  d, u, v = XMDC(a, n)
  
  if d !=1: 
    return False
  
  u %= n
  
  if u < 0: 
    u += n
    
  return u

criptografa = lambda b, e, n: (b**e) % n
descriptografa = lambda c, d, n: (c**d) % n 

In [4]:
from random import randint
# Demonstração:
# 1. Dois primos p e q
p = 269
q = 199

# 2. Calculando n = p * q e E entre {2, n-1} t. q. mdc(e, phi(n)) = 1
n = p * q
phi_n = (p-1)*(q-1)
e = 53 # pois mdc(n, 53) = 1
d = inverso_modular(e, phi_n)

phi_n

53064

## Alice Escreve uma mensagem, codifica e Criptografa

In [23]:
mensagem = "Oi" # Mensagem de Alice
mensagem_codificada = codifica_bloco(mensagem, 6)[0] #Mensagem codificada
mensagem_criptografada = criptografa(mensagem_codificada, e, n)

print(f"""Mensagem Original: {mensagem} 
Mensagem Codificada {mensagem_codificada}
Mensagem Criptografada {mensagem_criptografada}""")


Mensagem Original: Oi 
Mensagem Codificada 26959
Mensagem Criptografada 37220


## Bob Recebe a mensagem Criptografada e Descriptografa:

In [26]:
mensagem_descriptografada = descriptografa(mensagem_criptografada, d, n)
mensagem_decodificada = decodifica_bloco([mensagem_descriptografada])

print(f"Mensagem Descriptografada: {mensagem_descriptografada}\nMensagem Decodificada: {mensagem_decodificada}")

Mensagem Descriptografada: 26959
Mensagem Decodificada: ['O i ']


## Porquê isso funciona?

Provar que a criptografia RSA funciona é basicamente demonstrar que $D(C(b)) = b$. Provando isso, fica demonstrado que a criptografia funciona.
Primeiramente, é necessário verificar o seguinte resultado:
$$
se C(b) = b^e\ \ e\ \ D(x) = x^d\ (mod\ n) \therefore D(C(b)) = b^{ed}
$$
O que verificaremos será a seguinte equivalência:
$$
b^{ed} \equiv b\ mod\ n 
$$
Ou seja, vamos demonstrar que é possível reverter a mensagem criptografada para a mensagem original sem erros.
Por definição, temos que $b^{ed} \equiv b\ mod\ n$ é equivalente a dizer que $n\ | (b^{ed} - b)$. Como é dado que $n = pq$, o que deve ser provado, em realidade, é que: 

$$
  p\ | (b^{ed} - b)\ \ e \ \ q\ | (b^{ed} - b) 
$$

Se essa condição for cumprida, então a condição original do problema (ou seja, $b^{ed} \equiv b\ mod\ n$) é cumprida, e a criptografia funciona.

A demonstração é dividida em duas partes:
1. Se $p\ |\ b$, então $p\ |\ (b^{ed} - b)$ e a afirmação é verdadeira.
2. Caso contrário, é necessário relembrar do PTF, que diz que, caso $p$ seja um primo e não divida um número $b$, então $ b^{p-1} \equiv 1\ mod\ p $. Isso vai implicar que $ b^{p-1} \equiv 1\ mod\ p $. 
    1. Como $ ed \equiv 1\ mod\ \varphi(n) $, tem-se que
    $$
    ed = k\varphi(n)+1 = k(p-1)(q-1) + 1
    $$
    para algum $k$ inteiro.
    2. Então:
    $$
    b^{ed} = b^{k(p-1)(q-1)+1} = (b^{p-1})^{k(q-1)}*b \equiv b\ mod\ p
    $$
    Logicamente, $ b^{(p-1)} \equiv 1\ mod\ p$, logo, o resultado acima apenas expressa que $ b \equiv b\ mod\ p $, pois tudo anterior à multiplicação será 1. 